## download the data

```sh
pdm run data_imdb
```

In [1]:
import os
import pathlib
import random
import shutil

In [2]:
base_dir = pathlib.Path("./data/acl_imdb/aclImdb")


def prepare_val_data():
    val_dir = base_dir / "val"
    if val_dir.exists():
        print("validation data already existed")
        return
    train_dir = base_dir / "train"
    for category in ("neg", "pos"):
        os.makedirs(val_dir / category)
        files = os.listdir(train_dir / category)
        random.Random(1337).shuffle(files)
        num_val_samples = int(0.2 * len(files))
        val_files = files[-num_val_samples:]
        for fname in val_files:
            shutil.move(train_dir / category / fname, val_dir / category / fname)


prepare_val_data()

validation data already existed


In [3]:
from tensorflow import keras

batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    f"{base_dir}/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    f"{base_dir}/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    f"{base_dir}/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

Found 16000 files belonging to 2 classes.
Metal device set to: Apple M1 Max


2023-02-12 12:35:15.513194: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-12 12:35:15.513673: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Found 4000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


### Vectorizing the data

In [4]:
from tensorflow.keras import layers

CPU_CORES = 8

max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    # make the input size manageable, truncate after the first 600 words
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y), num_parallel_calls=CPU_CORES
)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y), num_parallel_calls=CPU_CORES
)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y), num_parallel_calls=CPU_CORES
)

2023-02-12 12:35:51.897110: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-12 12:35:51.937045: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


## Transformer encode

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(dense_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "num_heads": self.num_heads,
                "dense_dim": self.dense_dim,
            }
        )
        return config

### Using the Transformer encoder for text classification

In [6]:
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         5120000   
                                                                 
 transformer_encoder (Transf  (None, None, 256)        543776    
 ormerEncoder)                                                   
                                                                 
 global_max_pooling1d (Globa  (None, 256)              0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 257   

In [8]:
callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_encoder.keras", save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=5, callbacks=callbacks)
model = keras.models.load_model(
    "transformer_encoder.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder},
)
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/5
500/500 [==============================] - ETA: 0s - loss: 0.4313 - accuracy: 0.8003

2023-02-12 12:42:42.769878: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


500/500 [==============================] - 345s 689ms/step - loss: 0.4313 - accuracy: 0.8003 - val_loss: 0.3336 - val_accuracy: 0.8553
Epoch 2/5
500/500 [==============================] - 344s 689ms/step - loss: 0.3076 - accuracy: 0.8673 - val_loss: 0.3013 - val_accuracy: 0.8730
Epoch 3/5
500/500 [==============================] - 341s 682ms/step - loss: 0.2554 - accuracy: 0.8938 - val_loss: 0.2861 - val_accuracy: 0.8855
Epoch 4/5
500/500 [==============================] - 339s 677ms/step - loss: 0.2130 - accuracy: 0.9159 - val_loss: 0.3378 - val_accuracy: 0.8688
Epoch 5/5
500/500 [==============================] - 341s 682ms/step - loss: 0.1686 - accuracy: 0.9353 - val_loss: 0.2926 - val_accuracy: 0.8878


2023-02-12 13:05:56.430204: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - 177s 225ms/step - loss: 0.2868 - accuracy: 0.8787
Test acc: 0.879


## Using positional encoding to re-inject order information

### Implementing positional embedding as a subclassed layer

In [9]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim
        )
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "output_dim": self.output_dim,
                "sequence_length": self.sequence_length,
                "input_dim": self.input_dim,
            }
        )
        return config

### Combining the Transformer encoder with positional embedding

In [10]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 positional_embedding (Posit  (None, None, 256)        5273600   
 ionalEmbedding)                                                 
                                                                 
 transformer_encoder_1 (Tran  (None, None, 256)        543776    
 sformerEncoder)                                                 
                                                                 
 global_max_pooling1d_1 (Glo  (None, 256)              0         
 balMaxPooling1D)                                                
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                           

In [11]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "full_transformer_encoder.keras", save_best_only=True
    )
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=5, callbacks=callbacks)
model = keras.models.load_model(
    "full_transformer_encoder.keras",
    custom_objects={
        "TransformerEncoder": TransformerEncoder,
        "PositionalEmbedding": PositionalEmbedding,
    },
)
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/5


2023-02-12 13:34:18.746263: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


500/500 [==============================] - ETA: 0s - loss: 0.5094 - accuracy: 0.7544

2023-02-12 13:39:01.323622: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


500/500 [==============================] - 311s 620ms/step - loss: 0.5094 - accuracy: 0.7544 - val_loss: 0.2866 - val_accuracy: 0.8823
Epoch 2/5
500/500 [==============================] - 340s 680ms/step - loss: 0.2538 - accuracy: 0.8974 - val_loss: 0.4165 - val_accuracy: 0.8678
Epoch 3/5
500/500 [==============================] - 349s 699ms/step - loss: 0.1796 - accuracy: 0.9311 - val_loss: 0.3277 - val_accuracy: 0.8863
Epoch 4/5
500/500 [==============================] - 342s 684ms/step - loss: 0.1377 - accuracy: 0.9498 - val_loss: 0.3929 - val_accuracy: 0.8843
Epoch 5/5
500/500 [==============================] - 335s 670ms/step - loss: 0.1111 - accuracy: 0.9576 - val_loss: 0.5349 - val_accuracy: 0.8695


2023-02-12 14:02:16.041231: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - 176s 224ms/step - loss: 0.2920 - accuracy: 0.8782
Test acc: 0.878
